# Construcción de una Red Neuronal desde cero
A través de este tutorial, se va a construir una Red Neuronal Artificial (RNA) explicando de forma teórica los diferentes elementos necesarios para la construcción y entrenamiento del modelo. Además, se pondrá en práctica la red para evaluar su correcto funcionamiento.

# Índice de contenidos
- [1 - Inicialización de capas](#1)
- [2 - Inicialización de parámetros](#2)
- [3 - Implementar "forward propagation"](#3)
- [4 - Computación del coste](#4)
- [5 - Implementación del "backward propagation"](#5)
- [6 - Actualización de parámetros (gradient descent)](#6)
- [7 - Entrenamiento del modelo](#7)
- [8 - Predicción](#8)

# 1.- Instalación de librerías
En primer lugar, se instalan e importan todas las librerías necesarias para la realización de la implementación.

## 1.1.- Instalación de las librerías
Se va a emplear 'numpy' y 'matplotlib':

In [1]:
%pip install numpy
%pip install matplotlib


[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.6/162.6 kB 4.4 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.3/8.3 MB 34.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 306.0/306.0 kB 28.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 40.6 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 36.4 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 38.7 MB/s eta 0:00:0000:0100:01

[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [1]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

2024-08-03 15:58:56.557509: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-03 15:58:56.632749: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-03 15:58:56.653853: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-08-03 15:58:56.793643: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Num GPUs Available:  1


I0000 00:00:1722700739.267844    1655 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:07:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1722700739.321892    1655 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:07:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1722700739.321937    1655 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:07:00.0/numa_node
Your kernel may have been built without NUMA support.


## 1.2.- Inicialización de librerías
Una vez instaladas, se inicializan las librerías:

In [3]:
import numpy as np
import matplotlib as plt

# 2.- Inicialización de capas
Este método nos permite definir la estructura de la red neuronal que vamos a construir. 

In [4]:
def init_layers(X, Y, num_hidden_layers, num_hidden_units):
    """
    Este método permite definir la estructura de la red neuronal, especificando el número de neuronas en la capa de entrada (que viene dada por el tamaño del dataset utilizado),
    el número de capas ocultas especificado por parámetro de entrada, el número de neuronas en cada una de estas capas ocultas y finalmente, el número de neuronas en la capa de
    salida (especificado como el tamaño de la variable objetivo).

    Se entiende por tanto que se está construyendo una Red Neuronal Profunda donde tocas las capas ocultas tienen el mismo número de neuronas.

    Variables de entrada:
    X -- conjunto de datos de entrada (tamaño de entrada, número de ejemplos)
    Y -- etiquetas (tamaño de salida, número de ejemplos)
    
    Variables de salida:
    input_units -- número de neuronas en la capa de entrada (equivalente al número de variables de entrada)
    hidden_units -- número de neuronas en las capas ocultas
    output_units -- número de neuronas en la capa de salida
    """
    
    input_units = X.shape[0]
    hidden_layers = num_hidden_layers
    hidden_units = num_hidden_units 
    output_units = Y.shape[0]

    return (input_units, hidden_layers, hidden_units, output_units)

# 3.- Inicialización de parámetros
Este método nos permite inicializar los parámetros para una red neuronal de `l` capas.

In [ ]:
def init_parameters(dims_layers):
    """
    Esta función nos permitirá inicializar los parámetros de la Red Neuronal con la estructura definida en la función anterior.

    Variables de entrada:
    dims_layers -- consiste en las dimensiones de la red.

    Variables de salida
    parameters -- consiste en el conjunto de parámetros inicializados en la red
    """

    # Se crea el diccionario para almacenar los parámetros
    paramaters = {}

    for num_capa in range(1, dims_layers[hidden_layers] + 2):
        # Para cada capa se debe de crear la matriz de pesos y el sesgo correspondiente.
        paramaters['W' + str(num_capa)] = np.random.rand(dims_layers[hidden_units], dims_layers[hidden_units - 1])
        paramaters['b' + str(num_capa)] = np.random.rand(dims_layers[hidden_units], 1)

    # Devuelvo los parámetros inicializados
    return paramaters

# 4.- Implementación de 'forward propagation'
Para implementar la propagación hacia delante, se deben de realizar los siguientes cálculos:
* Cálculo de los parámetros pre-activación.
* Cálculo de la activación de las neuronas.


## 4.1.- Parámetros pre-activación
Este método se encarga de calcular los parámetros que se introducen en la función de activación de la neurona. La ecuación que se emplea es:
$$Z^{[l]} = W^{[l]}A^{[l-1]} +b^{[l]}\tag{4}$$
donde ${[l]}$ es el número de capa; $W^{[l]}$ son los pesos actuales de la capa actual; $A^{[l-1]}$ es la activación de la capa anterior; $b^{[l]}$ es el bias de la capa.

In [ ]:
def pre_activation(A, W, b):
    """_summary_
    Esta función permite calcular los parámetros pre-activacion de la neurona.
    
    Args:
        A (_type_): consiste en las activaciones obtenidas en la capa anterior.
        W (_type_): consiste en los pesos actuales de la capa.
        b (_type_): consiste en el sesgo actual de la capa.
    """

    # Obtengo los parámetros de pre-activación
    Z = np.dot(W,A) + b

    # Creo una tupla de Python guardada en cache para acelerar el entrenamiento de la red, ya que estos parámetros se usarán más en adelante
    pre_activation_params = (A, W, b)

    return Z, pre_activation_params

## 4.2.- Activación de la capa
Este método se encargará de calcular la activación de la neurona ante los parámetros pre-activación. Son múltiples las opciones que podemos emplear para realizar este paso. A continuación, explicamos y definimos las más comunes:
* **Sigmoid**: 
* **ReLU**:
* **Softmax**: 
* **Leaky ReLU**: 
* **Tanh**: 

### 4.2.1.- Sigmoid
La función sigmoidal sigue la siguiente fórmula matemática:


In [ ]:
def sigmoid(z):
    activation = 1 / (1 +  np.exp(-z))

    return activation

### 4.2.2.- ReLU
La función ReLU sigue la siguiente fórmula matemática:

In [ ]:
def relu(z):
    activation = max(0,z)

    return activation

### 4.2.3.- Softmax
La función Softmax sigue la siguiente fórmula matemática:

In [ ]:
def relu(z):
    max_z = np.max(z)
    exp = np.exp(z-max_z)
    activation = exp/max_z

    return activation

### 4.2.4.- Leaky ReLU
La función Leaky ReLU sigue la siguiente fórmula matemática:

In [ ]:
def leaky_relu(z):
    activation = max(0.01*z,z)

    return activation

### 4.2.5.- Tanh
La función Tanh sigue la siguiente fórmula matemática:

In [ ]:
def tanh(z):
    activation = np.tanh(z)

    return activation

# 5.- Computación de la pérdida
En este punto, ya tenemos computada la activación de cada capa por lo que tan sólo quedaría computar el coste. El coste define el error existente entre las predicciones del modelo y las etiquetas reales del conjunto de datos. Para asegurarnos de que nuestro modelo está aprendiendo correctamente, el coste debería de reducirse en cada etapa del entrenamiento.

Procedemos por tanto a definir algunas de las funciones de pérdida (que son las funciones que calculan el 'coste' sobre un conjunto de datos) más comunes:
* **Cross-entropy**: 
* **Log Loss**:

In [ ]:
def cost_function(predictions, Y):
    """_summary_
    Función que permite calcular el error entre las predicciones del modelo y las etiquetas correctas.

    Args:
        predictions (_type_): _description_
        Y (_type_): _description_
    """
    # Obtengo el número de ejemplos
    num_examples = Y.shape[1]

    # Calculo la función de pérdida
    cost = -(1/num_examples) * np.sum( Y * np.log(predictions) + (1-Y) * np.log(1-predictions))

    return cost

# 6.- Implementación del backward propagation

# 7.- Actualización de parámetros (Gradient Descent)

# 8.- Entrenamiento del modelo

# 9.- Predicción

# 10.- Conclusión